In [29]:
import pandas as pd
import os

In [30]:
def read_excel_files(folder_path):
    excel_dict = {}
    
    # Get a list of all files in the specified folder
    files = os.listdir(folder_path)
    
    # Iterate over each file in the folder
    for file in files:
        # Check if the file has an Excel extension (.xlsx or .xls)
        if file.endswith('.xlsx') or file.endswith('.xls'):
            file_path = os.path.join(folder_path, file)
            
            # Read the Excel file into a DataFrame
            df = pd.read_excel(file_path)
            
            # Add the DataFrame to the dictionary with the file name as the key
            excel_dict[file] = df
    
    return excel_dict

excel_dict = read_excel_files('../data')

print("Sheets in the Excel dict:")
for sheet_name in excel_dict.keys():
    print(sheet_name)

Sheets in the Excel dict:
Family History Repeat Group.xlsx
NPI Repeat Group.xlsx
Pulmonary Device Repeat Group.xlsx
Trial Details.xlsx
Surgery Repeat Group.xlsx
Assisstive Device Repeat Group.xlsx
Hospitalization Repeat Group.xlsx
DISC.xlsx
LOG.xlsx
DIAG.xlsx
ENCO.xlsx
Medication Repeat Group.xlsx
DEMO.xlsx
Clinvar Synmedical.xlsx


In [31]:
def remove_high_missing_columns(df, threshold=0.99):
    missing_percentages = df.isnull().mean()
    
    # Identify columns with missing values above the threshold
    columns_to_remove = missing_percentages[missing_percentages > threshold].index
    
    # Remove the identified columns from the DataFrame
    df = df.drop(columns=columns_to_remove)
    return df


demographics_df = remove_high_missing_columns(excel_dict['DEMO.xlsx'])
enco_df = remove_high_missing_columns(excel_dict['ENCO.xlsx'])
hospitalizations_df = remove_high_missing_columns(excel_dict['Hospitalization Repeat Group.xlsx'])
medications_df = remove_high_missing_columns(excel_dict['Medication Repeat Group.xlsx'])
diagnosis_df = remove_high_missing_columns(excel_dict['DIAG.xlsx'])
discontinuation_df = remove_high_missing_columns(excel_dict['DISC.xlsx'])
pulmonary_df = remove_high_missing_columns(excel_dict['Pulmonary Device Repeat Group.xlsx'])

In [32]:
medications_df_all = pd.read_csv('medications.csv')
medications_df_all = medications_df_all[medications_df_all['meds'] == 'Radicava']
radicava_users = medications_df_all[['FACPATID']]
medications_df_all

,FACPATID,medstdt,medstpdt,hltin,ethnic,meds,time passed,age_dx,gender
5,1072-10,2017-11-07,2017-11-17,Only Medicare,White,Radicava,10.0,23.1,Male
18,1100-4,2019-01-04,NaN,Only Medicare,White,Radicava,NaN,71.6,Female
20,1100-7,2018-10-01,NaN,Only Medicare,White,Radicava,NaN,82.0,Male
23,1100-9,2018-11-26,NaN,Only Medicare,White,Radicava,NaN,66.0,Male
29,1100-19,2019-01-01,NaN,Other,White,Radicava,NaN,58.0,Male
...,...,...,...,...,...,...,...,...,...
820,1441-55,2022-07-22,NaN,Private or group health insurance,White,Radicava,NaN,77.5,Male
823,1441-56,2022-08-03,NaN,Private or group health insurance,White,Radicava,NaN,70.0,Female
826,1441-6,2021-09-22,NaN,Private or group health insurance,White,Radicava,NaN,73.4,Male
827,1441-7,2021-11-19,NaN,Private or group health insurance,White,Radicava,NaN,56.9,Female


In [33]:
# Loaded variable 'medications_df' from kernel state

medications_df = pd.merge(radicava_users, medications_df[['FACPATID' , 'medname', 'medname1', 'medoth', 'medstdt', 'medstpdt']], on='FACPATID', how='inner')

medications_df['radicava'] = medications_df[['medname', 'medname1', 'medoth']].apply(lambda row: ('Radicava' in row.values), axis=1)
medications_df = medications_df[medications_df['radicava']]
medications_df = medications_df.drop(['medname', 'medname1', 'medoth'], axis = 1)

medications_df = medications_df[medications_df['medstdt'].notna()]

medications_df.reset_index(drop=True, inplace=True)


medications_df = medications_df[medications_df['medstdt'] != '08/03/2922 12:00:00 AM']
medications_df['medstdt'] = pd.to_datetime(medications_df['medstdt'])

medications_df['medstpdt'] = pd.to_datetime(medications_df['medstpdt'])

medications_df['time passed'] = (medications_df['medstpdt'] - medications_df['medstdt']).dt.days

/var/folders/_0/c6fzcd295qz6fty96376ybv00000gn/T/ipykernel_62433/2267872672.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  medications_df['medstdt'] = pd.to_datetime(medications_df['medstdt'])
/var/folders/_0/c6fzcd295qz6fty96376ybv00000gn/T/ipykernel_62433/2267872672.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  medications_df['medstpdt'] = pd.to_datetime(medications_df['medstpdt'])


In [34]:
medications_df = medications_df[medications_df['medstpdt'].notna()]

medications_df = medications_df[medications_df['time passed'] > 30]

In [ ]:
medications_df = pd.merge(medications_df, medications_df_all[['FACPATID', 'hltin']], on='FACPATID', how='inner')
medications_df.drop_duplicates(subset='FACPATID', keep='first', inplace=True)
medications_df

In [36]:
demographics_df

,FACPATID,CASE_ID,SCHEDULED_FORM_NAME,FORM_STATUS,Dataset,age,dstype,enroldt,gender,ethnic,...,nonmdapc,inschool,edulvl,employ,has_mname,dob1,sex,enroldt.P,dob.P,dob1.P
0,1048-1,1,Demographics,Complete,MOVR,47,ALS,2021-10-06,Male,Black or African American,...,Yes,No,Unknown,Unknown,NaN,03/20/1977 12:00:00 AM,M,day,day,day
1,1048-10,10,Demographics,Complete,MOVR,43,ALS,2021-12-08,Male,White,...,Yes,No,"Professional Degree (e.g., MD, DDS, DVM, JD)",Unknown,NaN,07/24/1980 12:00:00 AM,M,day,day,day
2,1048-11,11,Demographics,Complete,MOVR,84,ALS,2022-01-12,Male,White,...,Yes,No,Unknown,Retired,NaN,04/18/1939 12:00:00 AM,M,day,day,day
3,1048-19,19,Demographics,Complete,MOVR,76,ALS,2022-03-02,Male,White,...,Yes,No,Unknown,Full-time,NaN,04/23/1955 12:00:00 AM,M,day,day,day
4,1048-2,2,Demographics,Complete,MOVR,55,ALS,2021-10-06,Female,White,...,Unknown,Unknown,NaN,Full-time,NaN,03/04/1969 12:00:00 AM,F,day,day,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321,1446-5,5,Demographics,Complete,MOVR,79,ALS,2022-02-11,Female,White,...,Yes,No,Unknown,Retired,NaN,11/14/1944 12:00:00 AM,F,day,day,day
1322,1448-11,11,Demographics,Complete,MOVR,77,ALS,2022-02-24,Female,White,...,Yes,No,Unknown,Unknown,NaN,NaN,NaN,day,day,NaN
1323,1448-12,12,Demographics,Complete,MOVR,67,ALS,2021-09-22,Male,White,...,Yes,No,NaN,Retired,NaN,NaN,NaN,day,day,NaN
1324,9000-4,4,Demographics,Complete,MOVR,73,ALS,2020-04-24,Male,White,...,Yes,No,"Bachelor's Degree (e.g., BA, AB, BS, BBA)",Retired,NaN,NaN,NaN,day,day,NaN
